In [4]:
# Import libraries
import os
import boto3
import sagemaker

from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Preferences/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/saiomkarkandukuri/Library/Preferences/sagemaker/config.yaml


In [5]:
# initialize preconfigurations
region = boto3.Session().region_name
role = get_execution_role()

ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [ ]:
print(region)


In [ ]:
print(role)


In [ ]:
# Create S3 bucket

bucket = 'sagemaker-dallas-pca-ml'  #sagemaker.Session().default_bucket()

bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

print(bucket)
print(bucket_path)

In [ ]:
model_file_name = "dallas-local-RF-model"

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [ ]:
import joblib

mymodel = joblib.load(model_file_name)

In [ ]:
#import test file
import numpy as np

file_name = (
    "test_rf_sample.csv"  
)

with open(file_name, "r") as f:
    mypayload = np.loadtxt(f, delimiter=",")
    
print(mypayload)    

mymodel.predict(mypayload)

In [ ]:

# Create a tar.gz model file as this is the format required by Sagemaker for deployment.
mymodel._Booster.save_model(model_file_name)

In [ ]:
!tar czvf model.tar.gz $model_file_name


In [ ]:
# Upload the pre-trained model to S3

#### prefix in S3
prefix = "sagemaker/dallas-pca-ml"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

In [ ]:
## Set up hosting for the model
'''
Import model into hosting
This involves creating a SageMaker model from the model file previously uploaded to S3.
'''

In [ ]:
# Create a Sagemaker model
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")

In [ ]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

In [ ]:
# Create endpoint configuration
'''
Create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. 
In addition, the endpoint configuration describes the instance type required for model deployment.
'''
from time import gmtime, strftime

endpoint_config_name = "dallas-pca-ml-EndpointConfig" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
# Create endpoint
'''
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. 
The end result is an endpoint that can be validated and incorporated into production applications.
'''
import time
endpoint_name = "DEMO-XGBoost-Classifier-Endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
# Validate the model for use
'''
Now you can obtain the endpoint from the client library using the result from previous operations 
and generate classifications from the model using that endpoint.
'''
runtime_client = boto3.client("runtime.sagemaker")

In [ ]:
%%time
import json


file_name = (
    "test_point_classifier_sample.csv"  # customize to your test file
)

with open(file_name, "r") as f:
    payload = f.read().strip()
    
    
print("Payload :\n")

print(payload)
print()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

##print(response)

print("Results :\n")
print()

result = response["Body"].read().decode("utf-8")



In [ ]:
# Delete the Endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)
runtime_client.delete_endpoint(EndpointName=endpoint_name)